# Acceptance analysis of Direct Air Capture and Storage

Documentation of the analysis.

## Overview and correlation matrix

In [ ]:
from dacstore.utils import get_data, ensure_floats, get_data
from dacstore.config import encoding, replacer, weighting_groups, analyze_cols
from dacstore.dac_analysis import compute_group_averages, set_no_knowledge_to_neutral
import seaborn as sns
import matplotlib.pyplot as plt

cols = [
    "climate_change_perception",
    "tampering",
    "maturity",
    "benefit",
    "cost",
    "risk",
    "trust",
    "emotion",
    "distance",
    "dac_knowledge",
    "storage_knowledge",
    "initial_storage_support",
    "final_storage_support",
    "initial_dac_support",
    "final_dac_support",
    "age",
    "gender",
    "education",
    "occupation",
    "state",
]


y_col = "final_dac_support"

independent = [
    "final_dac_support",
    "final_storage_support",
    "initial_storage_support",
    "initial_dac_support",
]

# USER = "1392724-1415"
# PASSWORD = "it0pb4rmeggkl0dq4bk26k36s9wg4e7xydo5"


def create_numeric_data(df):
    """Create numeric data for regressions"""
    df = df.replace(replacer)
    df = ensure_floats(df, weighting_groups)
    df = compute_group_averages(df, weighting_groups)
    return df


# we work here with the original column names, no translation
df = get_data(
    source="../data/data.csv",
    drop=True,
    translate=False,
    drop_invalid=True,
    no_knowledge_to_neutral=True,
    set_dependent=True,
)

# an invalid entry (completed but more than 24hr completion time)
df = df.drop(91621021)

df = create_numeric_data(df)
df = df[weighting_groups.keys()]
df = df[~df.isna().any(axis=1)]
df = df[cols]

fig, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", ax=ax, annot_kws={"fontsize": 9})

## Creating the model

We load the data:

## Plots

Let's have a look at the filename.